# Routing

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI()

prompt = ChatPromptTemplate.from_template('''You are an elementary school math teacher
able to give very detailed and didactic answers. Answer the following student question:
Question: {question}''')
chain_math = prompt | model

prompt = ChatPromptTemplate.from_template('''You are an elementary school physics teacher
able to give very detailed and didactic answers. Answer the following student question:
Question: {question}''')
chain_physics = prompt | model

prompt = ChatPromptTemplate.from_template('''You are an elementary school history teacher
able to give very detailed and didactic answers. Answer the following student question:
Question: {question}''')
chain_history = prompt | model

prompt = ChatPromptTemplate.from_template('''{question}''')
chain_generic = prompt | model

c:\Projects\ai\langchain-playground\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1
c:\Projects\ai\langchain-playground\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
from pydantic import BaseModel, Field

prompt = ChatPromptTemplate.from_template('You must categorize the following question: {question}')

class Categorizer(BaseModel):
    """Categorizes elementary school student questions"""
    knowledge_area: str = Field(description='The knowledge area of the student question. \
    Must be "physics", "math" or "history". If it does not fit any of them, return "other"')

model_structured = prompt | model.with_structured_output(Categorizer)
model_structured.invoke({'question': 'When was the independence of the United States?'})

c:\Projects\ai\langchain-playground\.venv\Lib\site-packages\langchain_openai\chat_models\base.py:2073: UserWarning: Cannot use method='json_schema' with model gpt-3.5-turbo since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


Categorizer(knowledge_area='history')

### Creating routing structure

In [3]:
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough().assign(category=model_structured)
chain.invoke({'question': 'When was the independence of the United States?'})

{'question': 'When was the independence of the United States?',
 'category': Categorizer(knowledge_area='history')}

In [4]:
def route(input):
    if input['category'].knowledge_area == 'math':
        return chain_math
    if input['category'].knowledge_area == 'physics':
        return chain_physics
    if input['category'].knowledge_area == 'history':
        return chain_history
    return chain_generic

In [5]:
chain = RunnablePassthrough().assign(category=model_structured) | route
chain.invoke({'question': 'When was the independence of the United States?'})

AIMessage(content='The independence of the United States was declared on July 4, 1776. This significant event is known as the Declaration of Independence, which was written primarily by Thomas Jefferson and adopted by the Continental Congress. The declaration marked the formal break of the American colonies from British rule and laid the foundation for the establishment of the United States as a sovereign nation. The American Revolution, which followed the declaration, was a long and bloody conflict with Britain that ultimately resulted in the recognition of American independence with the signing of the Treaty of Paris in 1783. The Fourth of July holiday is celebrated annually to commemorate this historic day in American history.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 42, 'total_tokens': 169, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_pre

In [6]:
chain.invoke({'question': 'What is 1 + 21?'})

AIMessage(content='To find the sum of 1 + 21, we simply add the two numbers together. \n\n1\n+ 21\n-------\n  22\n\nTherefore, 1 + 21 equals 22.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 41, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-D7uQYLLocfuvEbqMDBaiYIGpqwDrG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c4a91-6f3e-78d2-9de9-2906f53e682c-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 41, 'output_tokens': 42, 'total_tokens': 83, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})